# Lab: Validation set and Cross-Validation
## CMSE 381 - Fall 2022
## Oct 2,  2023. Lecture 12



In [ ]:
# Everyone's favorite standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Validation set



The first thing we will do is try out the validation set approach. We've already used this some in class so this should be review.


## Validation set on toy data

First off, let's generate some toy data just to mess around. 

In [ ]:
# Set the seed so everyone has the same numbers
np.random.seed(42)

def f(t, m = -3, b = 2):
    return m*t+b

n = 300
X_toy = np.random.uniform(0,5,n)
y_toy = f(X_toy) + np.random.normal(0,2,n)

# reshaping to deal with cranky code later
X_toy = X_toy.reshape(-1,1)
y_toy = y_toy.reshape(-1,1)

plt.scatter(X_toy,y_toy)
plt.plot(X_toy,f(X_toy),c = 'red')

Ok, so now we have our fake data set up.  Extracting training and testing sets is as simple as the following single line.  

In [ ]:
from sklearn.model_selection import train_test_split
randomseed = 48824
X_train, X_test, y_train, y_test = train_test_split(X_toy,y_toy, 
                                                    test_size=0.2, 
                                                    random_state=randomseed)

What this does is extracts two pairs of input/output variables $X$ and $y$.  The train we will use to train our models, and then we will test (or validate) them on the test data. 

One way to see what these sets are is to plot them, although usually we have much higher $p$ (a.k.a. way more input variables) so we can't really visualize like this normally. 

In [ ]:
plt.scatter(X_train,y_train, marker = '+', label = "Training")
plt.scatter(X_test,y_test, marker = '*', label = "Testing")
plt.legend()

&#9989; **<font color=red>Do this:</font>** Set up a linear regression model, train it on the training set, and test it on the test set.  What is your mean squared error? 

In [ ]:
# Your code here

Now that we can see what happens in one case, let's try doing this many times.  
The code below changes our random seed. 

&#9989; **<font color=red>Do this:</font>** Copy the code needed to generate a train/test split with the new seed and compute the MSE. 

&#9989; **<font color=red>Q:</font>** What happens to the MSE? 


In [ ]:
# Running this changes the random seed to get a new split of the data
randomseed +=1


# Put your code here to generate a new train test split, run a new model, and compute the MSE


&#9989; **<font color=red>Do this:</font>**  Run the above script in a loop, repeating $k=30$ times. Keep track of the MSE in a list and draw a histogram of the results. What do you notice? If you want to see more pattern, set $k$ to be something larger like 100.

In [ ]:
# Your code here




![Stop Icon](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Vienna_Convention_road_sign_B2a.svg/180px-Vienna_Convention_road_sign_B2a.svg.png)

Great, you got to here! Hang out for a bit, there's more lecture before we go on to the next portion. 

# 2. Leave One Out Cross Validation (LOOCV)

Luckily, `sklearn` has a simple built in procedure to extract your LOOCV splits for easily passing to your models. However, these work a bit differently than before. As always, the `sklearn` documentation and user guide is an excellent place to start. 

- [LOOCV Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html)
- [LOOCV User guide](https://scikit-learn.org/stable/modules/cross_validation.html#leave-one-out)

In [ ]:
# I'm copying the same code from above just so we're restarting with the same data. 


# Set the seed so everyone has the same numbers
np.random.seed(42)

def f(t, m = -3, b = 2):
    return m*t+b

n = 300
X_toy = np.random.uniform(0,5,n)
y_toy = f(X_toy) + np.random.normal(0,2,n)

# reshaping to deal with cranky code later
X_toy = X_toy.reshape(-1,1)
y_toy = y_toy.reshape(-1,1)

plt.scatter(X_toy,y_toy)
plt.plot(X_toy,f(X_toy),c = 'red')

In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
loo.get_n_splits(X_toy)

# Notice that trying to print loo doesn't give us much that's useful
print(loo)

In [ ]:
# The power of the function shows up when we use it in a for loop:
for train_index, test_index in loo.split(X_toy):
    print("TRAIN:", train_index, "TEST:", test_index)
    

&#9989; **<font color=red>Question:</font>** What's the difference between the `LOO.split` output and the `train_test_split` from before?

Your answer here.

The code below fixes the problem, and draws a figure like above where we get to see which data point is part of the testing set. 

In [ ]:
# These generator objects are a bit weird, so I can't just pull one split
# out of the list. The code below is a bit convoluted, but essentially I'm 
# just trying to draw the last train/test split found. 
countPrint = 0
maxPrint = 7 # If you change this number, a different point will be labeled as 
             # the testing point because I make the loop stop at maxPrint.

for train_index, test_index in loo.split(X_toy):

    countPrint +=1 

    X_train, X_test = X_toy[train_index], X_toy[test_index]
    y_train, y_test = y_toy[train_index], y_toy[test_index]
    
    # Uncomment if you want to see how this fixes it
#     print(X_train,y_train)
#     print(X_test, y_test)
#     print('\n')

    if countPrint >= maxPrint:
        break

plt.scatter(X_train,y_train, marker = '+', label = "Training")
plt.scatter(X_test,y_test, marker = '*', label = "Testing")
plt.legend()

&#9989; **<font color=red>Do this:</font>** Use the leave one out splits to perform a linear regression on each one, return the mean squared error, and then average over all the values to get the LOOCV error estimation.

What do you notice about this error estimation vs. the validation set version above? 


In [ ]:
# Your code here #

## The easier version

Ok I lied to you a bit. There's an even easier version of this whole LOO-CV thing. 

In [ ]:
# This command does all your work for you
from sklearn.model_selection import cross_val_score


#define cross-validation method to use
cv = LeaveOneOut()

#build linear regression model
model = LinearRegression()

#use LOOCV to evaluate model
scores = cross_val_score(model, X_toy, y_toy, 
                         scoring='neg_mean_squared_error',
                         cv=cv)

#view mean absolute error
np.average(np.absolute(scores))



Check out this number. Remember LOO-CV has no randomness in it so you should have the same number here as you calculated previously. 



-----
### Congratulations, we're done!
Written by Dr. Liz Munch, Michigan State University

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.